# 矩阵乘法
在本教程中，您将编写一个非常简短的高性能 FP16 矩阵乘法内核，其性能可与 cuBLAS 或 rocBLAS 媲美。
您将具体了解：
- 块级矩阵乘法。
- 多维指针运算。
- 程序重新排序以提高 L2 缓存命中率。
- 自动性能调整。

## 动机
矩阵乘法是大多数现代高性能计算系统的关键组成部分。众所周知，矩阵乘法难以优化，因此其实现通常由硬件供应商自行完成，作为所谓的“内核库”（例如 cuBLAS）的一部分。遗憾的是，这些库通常是专有的，无法轻松定制以适应现代深度学习工作负载（例如融合激活函数）的需求。在本教程中，您将学习如何使用 Triton 自行实现高效的矩阵乘法，并且易于定制和扩展。

粗略地说，我们将要编写的内核将实现以下分块算法，将 (M, K) 乘以 (K, N) 矩阵：

In [ ]:
# Do in parallel
for m in range(0, M, BLOCK_SIZE_M):
  # Do in parallel
  for n in range(0, N, BLOCK_SIZE_N):
    acc = zeros((BLOCK_SIZE_M, BLOCK_SIZE_N), dtype=float32)
    for k in range(0, K, BLOCK_SIZE_K):
      a = A[m : m+BLOCK_SIZE_M, k : k+BLOCK_SIZE_K]
      b = B[k : k+BLOCK_SIZE_K, n : n+BLOCK_SIZE_N]
      acc += dot(a, b)
    C[m : m+BLOCK_SIZE_M, n : n+BLOCK_SIZE_N] = acc

其中双重嵌套 for 循环的每次迭代均由专用的 Triton 程序实例执行。

## 计算内核

实际上，上述算法在 Triton 中实现起来相当简单。主要的困难在于计算在内循环中必须读取 A 和 B 块的内存位置。为此，我们需要多维指针运算。

### 指针运算

对于行主二维张量 `X` ， `X[i, j]` 的内存位置由 `&X[i, j] = X + i*stride_xi + j*stride_xj` 给出。因此， `A[m : m+BLOCK_SIZE_M, k:k+BLOCK_SIZE_K]` 的指针块和 `B[k : k+BLOCK_SIZE_K, n : n+BLOCK_SIZE_N]` 可以用伪代码定义为：

```
&A[m : m+BLOCK_SIZE_M, k:k+BLOCK_SIZE_K] =  a_ptr + (m : m+BLOCK_SIZE_M)[:, None]*A.stride(0) + (k : k+BLOCK_SIZE_K)[None, :]*A.stride(1);
&B[k : k+BLOCK_SIZE_K, n:n+BLOCK_SIZE_N] =  b_ptr + (k : k+BLOCK_SIZE_K)[:, None]*B.stride(0) + (n : n+BLOCK_SIZE_N)[None, :]*B.stride(1);
```

这意味着可以在 Triton 中初始化 A 和 B 块的指针（即 `k=0` ），代码如下。另请注意，我们需要一个额外的模来处理 `M` 和 `N` 如果 `M` 不是 `BLOCK_SIZE_M` 的倍数 或 `N` 不是 `BLOCK_SIZE_N` 的倍数，我们可以用一些无用的值填充数据，但这些值不会对结果产生影响。对于 K 维度，我们稍后会使用屏蔽加载语义来处理。

```
offs_am = (pid_m * BLOCK_SIZE_M + tl.arange(0, BLOCK_SIZE_M)) % M
offs_bn = (pid_n * BLOCK_SIZE_N + tl.arange(0, BLOCK_SIZE_N)) % N
offs_k = tl.arange(0, BLOCK_SIZE_K)
a_ptrs = a_ptr + (offs_am[:, None]*stride_am + offs_k [None, :]*stride_ak)
b_ptrs = b_ptr + (offs_k [:, None]*stride_bk + offs_bn[None, :]*stride_bn)
```

然后在内循环中更新如下：

```
a_ptrs += BLOCK_SIZE_K * stride_ak;
b_ptrs += BLOCK_SIZE_K * stride_bk;
```